In [1]:
 !wget "https://drive.google.com/uc?export=download&id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W" -O E-commerce.csv

--2024-10-02 05:20:10--  https://drive.google.com/uc?export=download&id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W
Resolving drive.google.com (drive.google.com)... 142.251.2.101, 142.251.2.139, 142.251.2.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W&export=download [following]
--2024-10-02 05:20:10--  https://drive.usercontent.google.com/download?id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.137.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139827 (137K) [application/octet-stream]
Saving to: ‘E-commerce.csv’

E-commerce.csv      100%[===================>] 136.55K  --.-KB/s    i

# Imports

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
# from sklearn.decomposition import PCA

In [2]:

df=pd.read_csv("E-commerce.csv")
df.head()


,ID,n_clicks,n_visits,amount_spent,amount_discount,days_since_registration,profile_information
0,1476,130,65,213.905831,31.600751,233,235
1,1535,543,46,639.223004,5.689175,228,170
2,1807,520,102,1157.402763,844.321606,247,409
3,1727,702,83,1195.903634,850.041757,148,200
4,1324,221,84,180.754616,64.283300,243,259


In [10]:
df=df.drop(['ID'], axis=1)
df.head()

,n_clicks,n_visits,amount_spent,amount_discount,days_since_registration,profile_information
0,130,65,213.905831,31.600751,233,235
1,543,46,639.223004,5.689175,228,170
2,520,102,1157.402763,844.321606,247,409
3,702,83,1195.903634,850.041757,148,200
4,221,84,180.754616,64.283300,243,259


# KMeans
## Steps
**1. Initialization**
  -  Randomly initialize k centers from the dataset.

**2. Assignment**
  - For each point, we find the distance of existing centroids
from it and assign the point to that cluster whose centroid has the
minimum distance.

**3. Update**
  - Update the centroids of the clusters by taking the average of points from each cluster.

**4. Repeat**
  - Repeat the previous two steps until convergence (the center of new
cluster centroids stops changing their positions).

## Prepare data

In [118]:
X=df.to_numpy()
scaler=StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
X.shape, X[0,:], X.dtype

((2500, 6),
 array([-1.49525046, -0.75853514, -1.05461141, -0.73280039,  0.32311781,
         0.33919174]),
 dtype('float64'))

In [119]:
k = 4

In [146]:
# 1step initialize K centroids
indexes = np.random.choice(X.shape[0],size=k)
centroids = X[indexes]
indexes, centroids

(array([1484, 1374, 1532]),
 array([[-0.9157792 , -0.88720684, -0.06121561, -0.736285  , -0.48401084,
          1.03835846],
        [-0.49190671, -0.11517662, -0.52156792, -0.1362648 ,  0.03053368,
          0.47902508],
        [-0.29874962,  1.45461818, -0.81505647, -0.47392221,  1.2916722 ,
          0.54894176]]))

In [121]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity, manhattan_distances


In [122]:
# prompt: create euclidean distance between matrix and centroids array

import numpy as np
distances = np.sqrt(np.sum((X[:, np.newaxis, :] - centroids[np.newaxis, :, :]) ** 2, axis=2))
distances.shape

(2500, 3)

In [145]:
# 2step
# For each point, we find the distance of existing centroids from it
# and assign the point to that cluster whose centroid has the minimum distance.
def get_distance(centroids, X,metric='euclidean'):
  if metric == 'euclidean':
    distances = euclidean_distances(centroids, X)
  elif metric == 'cosine':
    distances = cosine_similarity(centroids, X)
  elif metric == 'manhattan':
    distances = manhattan_distances(centroids, X)
  else:
    raise ValueError('Invalid metric')
  return distances

In [140]:
distances[1].shape # distance from 1st centroid to every point

assignment = np.argmin(distances, axis=0)

# new assignment
new_centroids = np.array([X[assignment == i].mean(axis=0) for i in range(k)])
new_centroids

# if np.array_equal(centroids, oldCentroids) :
#     return membership, centroids


(3, 6)

In [185]:
iterations = 0
oldCentroids = None
MAX_ITERATIONS = 100
assignment=None
k=4
# 1step initialize K centroids
indexes = np.random.choice(X.shape[0],size=k)
centroids = X[indexes]
indexes, centroids
while not np.array_equal(centroids, oldCentroids) and iterations < MAX_ITERATIONS:
  oldCentroids = centroids

  distances = get_distance(centroids, X)
  assignment = np.argmin(distances, axis=0)
  centroids = np.array([X[assignment == i].mean(axis=0) for i in range(k)])
  if np.array_equal(centroids, oldCentroids):
    break
  print(np.array_equal(centroids, oldCentroids))
  print(iterations)
  iterations += 1


False
0
False
1
False
2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
10
False
11
False
12
False
13
False
14
False
15


In [186]:
df.loc[assignment==0,'cluster']=0
df.loc[assignment==1,'cluster']=1
df.loc[assignment==2,'cluster']=2
df.loc[assignment==3,'cluster']=3
df.head()

,n_clicks,n_visits,amount_spent,amount_discount,days_since_registration,profile_information,cluster
0,130,65,213.905831,31.600751,233,235,0
1,543,46,639.223004,5.689175,228,170,0
2,520,102,1157.402763,844.321606,247,409,2
3,702,83,1195.903634,850.041757,148,200,2
4,221,84,180.754616,64.283300,243,259,0


In [187]:
df.groupby('cluster').mean()

,n_clicks,n_visits,amount_spent,amount_discount,days_since_registration,profile_information
cluster,,,,,,
0,374.618541,92.677812,719.431651,180.665189,287.907295,205.767477
1,250.583804,67.487759,3339.325173,82.559790,196.794727,215.190207
2,601.716346,128.578526,1409.191746,1127.415732,199.956731,198.607372
3,388.165939,86.081514,708.622301,152.907181,121.863173,187.786026
